# Policy Gradient

## 이전 시간

- value based reinforcement learning을 했고, Q라는 action value function을 이용해서 Q function을 구하고 Q function을 통해서 policy를 찾았다.
- DQN도 value based RL이고 DNN으로 Q function을 approximate하고 그것으로 policy를 찾았다.

## policy gradient 장/단점

- 장점
  - 수렴이 더 잘된다.
  - action이 연속적인 경우에 효과적이다. value based function은 discrete한 output에 대해서 다루었다.
  - 확률적인 policy를 배울 수 있다. ex) 가위바위보 게임
- 단점
  - local minima에 빠지기 쉽다.
  - policy의 evaluation이 비효율적이다.
  - variance가 높다.

## value based RL의 문제

- value function을 바탕으로 policy를 계산하기 때문에 value function이 조금만 달라져도 policy에 크게 영향을 받는다. 이 현상은 알고리즘ㅈ 수렴에 불안정을 더한다.
- stochastic policy
  - 때로는 stochastic policy가 최적의 policy가 될 수 있다.

## Objective function

- Q function을 사용하는 action value based 방법에서는 DNN을 update 하기위해서 target과 현재 Q function값의 MSE를 구했다.
  - Q function -> Q function이 나오는 DNN
  - policy -> policy가 나오는 DNN
<br>
- Objective function
  1. state value, 게임에서 보통 똑같은 state에서 시작하기 때문에 처음 시작 state의 value function이 최대로 되게 학습한다.
  2. average value
  3. average reward per time-step, 각 time-step 마다 받는 reward를 각 state에서 머무르는 비율(stationary distribution)을 곱한 expectation 사용
<br>
- 따라서, policy gradient의 목표는 이 objective function을 최대화 시키는 theta라는 policy의 parameter vector를 찾는것이다.
  - how? gradient descent


## Objective function 구하는 법

- Finite difference policy gradient
  - parameter vector가 5개의 차원으로 이루어져 있으면 하나씩 흔들어 보면서 그 gradient를 구한다. policy가 미분가능하지 않아도 사용할 수 있는 방법
- Monte carlo policy gradient
  - objective function에 직접 gradient를 취해준다. policy는 미분 가능하다고 본다.
- actor critic policy gradient

## score function

\\( \begin{split}
\bigtriangledown_{\theta} \pi_{\theta}(s,a) &= \pi_{\theta}(s,a) \frac{\bigtriangledown_{\theta} \pi_{\theta}(s,a)}{\pi_{\theta}(s,a)} \\
&= \pi_{\theta}(s,a) \bigtriangledown_{\theta} \log \pi_{\theta}(s,a) \end{split} \\)

\\(\begin{split} J(\theta)
&= \mathbb{E}_{\pi_{\theta}}[r] \\
&= \sum_{s \in \mathcal{S}}d(s) \sum_{a \in \mathcal{A}} \pi_\theta(s,a)R_{s,a}
\end{split}\\)
<br><br>
\\(\begin{split} \bigtriangledown_{\theta} J(\theta)
&= \sum_{s \in \mathcal{S}}d(s) \sum_{a \in \mathcal{A}} \pi_\theta(s,a) \bigtriangledown_{\theta} \log \pi_{\theta}(s,a)R_{s,a} \\
&= \mathbb{E}_{\pi_\theta}[\bigtriangledown_{\theta} \log \pi_{\theta}(s,a)r]
\end{split}\\)

- 위 마지막 식에서 정의된 \\(r\\)은 immediate reward이다.
- \\(\bigtriangledown_{\theta} J(\theta)\\)를 이용해서 policy DNN update한다.
- 그러나 강화학습은 stochastic policy gradient로써 policy 자체가 stochastic하다.
  - policy 2가지 표현, softmax policy, gaussian policy

## Monte carlo policy gradient

- episode가 끝나면 해당 policy에 대한 Q function을 구할 수 있다. 하지만 episode가 너무 길면?
- 높은 variance를 가진다.
  - 위 문제를 해결하기 위해서 Neural network를 2개 만들어서 Q function도 approximate해서 gradient를 구하자 -> actor critic policy gradient

## Actor critic policy gradient

- Actor, policy approximate하고,
- critic, Q function approximate한다. -> 현재 policy를 평가한다.
  - 따라서 w와 \\(\theta\\) 두개를 사용한다.

- policy evaluation
  - Monte carlo policy evaluation
  - TD learning
  - TD(\\(\lambda \\))

## Baseline

- Actor critic에 baseline을 추가하여 variance 문제를 해결한다.
- 예를 들어) action 1를 했을 때 reward를 100만을 받고, action 2를 했을 때 reward 99만을 받는다면 결국 수렴하게 되겠지만 variance가 너무 크다 따라서 action value function에서 state value function을 빼주어 advantage를 만들어 variance문제를 해결한다.